In [9]:
from langchain.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

ollama = Ollama(base_url="http://localhost:11434", model="autopilot")

TEXT_PROMPT1 = "who are you?"
TEXT_PROMPT2 = "what can you do?"
from pprint import pprint
pprint(ollama(TEXT_PROMPT1))
pprint(ollama(TEXT_PROMPT2))


('I am Open-Assistant, a chatbot based on a large language model. You can ask '
 'me anything, I will do my best to provide you helpful answers.')
('I can help you write a Golang function to reconcile the version of a '
 'Kubernetes Deployment for a container App.')


In [35]:

# Define the template with placeholders

template = """

You are an expert in Kubernetes development. You will use Operator SDK as the guide to aid in the development of a Kubernetes operator. The operator will be used to deploy a containerized application. The application will be deployed using the following details:

The operator will deploy custom resource: {resource}

with the pod image: {image}

Output step by step instruction on how to develope and deploy Kubernetes operator using Operator SDK.:

"""

# Create a PromptTemplate object

prompt_template = PromptTemplate(template=template, input_variables=["resource", "image"]) 

# Example article details

custom_resource = "rose"

operand_image = """

nginx:latest
"""

# Fill the template with the article details

prompt = prompt_template.format(resource=custom_resource, image=operand_image)

# Get the summary from the language model

def chat_with(llm, prompt):
    return llm.invoke(prompt) 

ollama = Ollama(base_url="http://localhost:11434", model="autopilot")

summary = chat_with(ollama,prompt)    

print(summary)       

To develop a Kubernetes operator, you can use the Operator SDK. Here is a step-by-step guide on how to do this:

1. Install Operator SDK: Follow the installation instructions in the Operator SDK documentation.
2. Create a new project: Use the following command to create a new project for your operator:

```bash
$ cd ${HOME}
$ mkdir sample-operator && cd $_
$ operator-sdk init --domain sample.com --owner "The Sample Operator Team <sample-operator@email.com>"
$ go mod vendor
```

3. Add custom resource definition (CRD): Create a new file called `bases/rose.yaml` and add the following content:

```yaml
apiVersion: apiextensions.k8s.io/v1beta1
kind: CustomResourceDefinition
metadata:
  name: roses.samples.com
spec:
  group: samples.com
  names:
    kind: Rose
    listKind: RoseList
    plural: roses
    singular: rose
  scope: Namespaced
  version: v1alpha1
```

4. Implement controller logic: Create a new file called `controllers/rose_controller.go` and add the following content:

```go
pa

In [48]:
from langchain.chains import LLMChain

# Define a prompt template
chain1_prompt_template = PromptTemplate(template="{text}", input_variables=["text"]) 

chain2__prompt = PromptTemplate(
    input_variables=["joke"], 
    template="First, Explain the following joke: {joke}, Second,then tell me a better joke."
)

chain3_prompt_template = PromptTemplate(
    input_variables=["joke1", "joke2"],
    template="Can you tell me if joke1: {joke1}, is better than joke2: {joke2}"
)      

# Define the chain
chain1 = LLMChain(
    llm=ollama,
    prompt=prompt_template,
    output_key="joke1"
)
chain2 = LLMChain(
    llm=ollama,
    prompt=prompt_template,
    output_key="joke2"
)
chain3 = LLMChain(
    llm=ollama,
    prompt=chain3_prompt_template,
    output_key="critique"
) 
        

In [50]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[
        chain1,
        chain2,
        chain3
        
    ],
    input_variables=["text"],
    output_variables = ["joke1","joke2", "critique"],
    verbose=True
    
)

In [51]:
result = chain({'text': 'tell me a joke'})
pprint(result)        



> Entering new SequentialChain chain...

> Finished chain.
{'critique': 'Both are jokes, but I would say that joke1 is more surprising '
             'and unexpected. In joke1, the element of surprise comes from the '
             'unusual pairing of an elephant and a blue whale, while in joke2, '
             'the surprise comes from the chicken actually crossing the '
             'playground.',
 'joke1': "Here's one, what do you get when you cross an elephant and a blue "
          'whale?',
 'joke2': 'Why did the chicken cross the playground? To get to the other '
          'slide.',
 'text': 'tell me a joke'}


In [52]:
from langchain_community.document_loaders import WebBaseLoader

# List of URLs you want to load (We will crawl the entire site later)
urls = [
    "https://docs.nvidia.com",
    "https://docs.nvidia.com/cuda",
    "https://docs.nvidia.com/deeplearning",
    "https://docs.nvidia.com/gameworks"
    "https://docs.nvidia.com/cudnn",
    "https://docs.nvidia.com/tensorrt",
    
    
]

data = []

# Loop through each URL and load the page content
for url in urls:
    loader = WebBaseLoader(url)
    page = loader.load()
    page[0].page_content = page[0].page_content.replace('\n', '')
    data.extend(page)
    

pprint(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://docs.nvidia.com', 'title': 'NVIDIA Documentation Hub - NVIDIA Docs', 'description': 'Get started by exploring the latest technical information and product documentation', 'language': 'en'}, page_content="NVIDIA Documentation Hub - NVIDIA DocsSubmit SearchNVIDIA DeveloperBlogForumsJoinSubmit SearchNVIDIA DeveloperBlogForumsJoinMenuNVIDIA Documentation HubGet started by exploring the latest technical information and product documentationBrowse byFeaturedProductsAll DocumentsSubmit SearchMost PopularGet support for our latest innovations and see how you can bring them into your own work.NVIDIA API Documentation        Your guide to NVIDIA APIs including NIM and CUDA-X microservices.                Browse            NVIDIA AI Enterprise        NVIDIA AI Enterprise is an end-to-end, cloud-native software platform that accelerates data science pipelines and streamlines development and deployment of production-grade co-pilots and other generative AI appl

In [53]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_docs(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    return text_splitter.split_documents(data)


In [69]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")        


In [75]:
# Add to ChromaDB vector store
#from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma

# Define the variable "all_splits"
all_splits = split_docs(data)

vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()


AttributeError: 'Collection' object has no attribute 'model_fields'

In [88]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("state-of-union.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

AttributeError: 'Collection' object has no attribute 'model_fields'

In [87]:
pip show chromadb

Name: chromadb
Version: 0.4.24
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /Users/rosecrisp/anaconda3/lib/python3.11/site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pulsar-client, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: langchain-chroma
Note: you may need to restart the kernel to use updated packages.
